In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
from operator import itemgetter
import copy
import numpy as np

Nous nous focalisons ici sur l'algorithme permettant de trouver le k-core contenant les mots-clefs.

# Python

## Algorithme en Python "simple"

In [3]:
def elbow_python(sorted_scores):
    sorted_scores = [np.array(i) for i in sorted_scores]

    first_point = sorted_scores[0]
    last_point =sorted_scores[-1]
    distances = []

    for index,point in enumerate(sorted_scores):
        point= np.array(point)
        d=np.cross(last_point-first_point,point-first_point)/np.linalg.norm(last_point-first_point)
        distances.append(abs(d))

    if np.max(distances)>0:
        x_elbow=sorted_scores[np.argmax(distances)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow


def get_density(g):
    e = g.number_of_edges()
    v = g.number_of_nodes()

    if v !=0:
        density = abs(e)/(abs(v)*(abs(v)-1))
        return density
    else : 
        return None

def dens_method(g,k_core_decomp):
    D=[]
    for i in range(max(k_core_decomp.values())):
        nodes_indexes = [k for k, v in k_core_decomp.items() if v >i]
        g_small = g.subgraph(nodes_indexes)
        D.append(get_density(g_small))
    return elbow([i for i in enumerate(D)])[0]+1 # +1 as we started counting at zero

def get_keywords(g):
    k_core_decomp = nx.core_number(g) # replace by custom made function
    k_core_value_of_keywords = dens_method(g,k_core_decomp)
    keywords = [k for k, v in k_core_decomp.items() if v >= k_core_value_of_keywords]
    return keywords


# Cython

In [4]:
%load_ext Cython

Pour la suite, et afin de pouvoir appliquer Cython de façon approfondie sur une tâche simple, nous nous focalisons sur la fonction permettant de trouver le coude (elbow_python dans l'exemple Python "simple")

## Cython naïf

In [5]:
%%cython
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange

def naive_cython_elbow(sorted_scores):
    sorted_scores = [np.array(i) for i in sorted_scores]

    first_point = sorted_scores[0]
    last_point =sorted_scores[-1]
    distances = []

    for index,point in enumerate(sorted_scores):
        point= np.array(point)
        d=np.cross(last_point-first_point,point-first_point)/np.linalg.norm(last_point-first_point)
        distances.append(abs(d))

    if np.max(distances)>0:
        x_elbow=sorted_scores[np.argmax(distances)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow

## en remplaçant des fonctions numpy par des cdef (et en ajoutant les types)

In [6]:
%%cython
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange


cdef float cross_prod_vect( first_point, last_point,  point):
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef float Bx = (point[0]-first_point[0])
    cdef float By = (point[1]-first_point[1])
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm
    

cdef float cross_prod(float Ax, float Ay, float Bx, float By):
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm

cdef float density_with_e_v(float e, float v):
    return abs(e)/(abs(v)*(abs(v)-1))   

cpdef cdef_elbow(sorted_scores):

    cdef int elbow_point
    cdef double[:] first_point = sorted_scores[0]
    cdef double[:] last_point = sorted_scores[-1]
    cdef int i # pour optimiser la boucle
    cdef double[:] Y = np.zeros(len(sorted_scores))

    for i in range(len(sorted_scores)):
        point = sorted_scores[i]
        d = cross_prod_vect(first_point,last_point,point)
        Y[i]=abs(d)
        
    if np.max(Y)>0:
        x_elbow=sorted_scores[np.argmax(Y)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow

# en ajoutant les memoryviews

In [7]:
%%cython
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange


cdef float cross_prod_vect(double[:] first_point, double[:] last_point,  double[:] point):
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef float Bx = (point[0]-first_point[0])
    cdef float By = (point[1]-first_point[1])
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm
    

cdef float cross_prod(float Ax, float Ay, float Bx, float By):
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm

cdef float density_with_e_v(float e, float v):
    return abs(e)/(abs(v)*(abs(v)-1))   

cpdef c_elbow_memoryviews(double[:, :] sorted_scores):

    cdef int elbow_point
    cdef double[:] first_point = sorted_scores[0]
    cdef double[:] last_point = sorted_scores[-1]
    cdef int i # pour optimiser la boucle
    cdef double[:] Y = np.zeros(len(sorted_scores))

    for i in range(len(sorted_scores)):
        point = sorted_scores[i]
        d = cross_prod_vect(first_point,last_point,point)
        Y[i]=abs(d)
        
    if np.max(Y)>0:
        x_elbow=sorted_scores[np.argmax(Y)]
    else :
        x_elbow=sorted_scores[0]
        

    return x_elbow


## en desactivant boundscheck, wraparound 
(nous devons faire en sorte qu'aucun index ne soit alors négatif)

In [8]:
%%cython
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange


cdef float cross_prod_vect(double[:] first_point, double[:] last_point,  double[:] point):
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef float Bx = (point[0]-first_point[0])
    cdef float By = (point[1]-first_point[1])
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm
    

cdef float cross_prod(float Ax, float Ay, float Bx, float By):
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm

cdef float density_with_e_v(float e, float v):
    return abs(e)/(abs(v)*(abs(v)-1))   

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef c_elbow_noboundscheck_nowraparound(double[:, :] sorted_scores):

    cdef int elbow_point
    cdef int N = sorted_scores.shape[0]
    cdef double[:] first_point = sorted_scores[0]
    cdef double[:] last_point = sorted_scores[N-1]
    cdef int i # pour optimiser la boucle
    cdef double[:] Y = np.zeros(N)

    for i in range(N):
        point = sorted_scores[i]
        d = cross_prod_vect(first_point,last_point,point)
        Y[i]=abs(d)
        
    if np.max(Y)>0:
        x_elbow=sorted_scores[np.argmax(Y)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow

# en déclarant les np arrays comme  continus

In [29]:
%%cython
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange


cdef float cross_prod_vect(double[::1] first_point, double[::1] last_point,  double[::1] point):
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef float Bx = (point[0]-first_point[0])
    cdef float By = (point[1]-first_point[1])
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm
    

cdef float cross_prod(float Ax, float Ay, float Bx, float By):
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm

cdef float density_with_e_v(float e, float v):
    return abs(e)/(abs(v)*(abs(v)-1))   

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef c_elbow_continous(double[:, ::1] sorted_scores):

    cdef int elbow_point
    cdef int N = sorted_scores.shape[0]
    cdef double[::1] first_point = sorted_scores[0]
    cdef double[::1] last_point = sorted_scores[N-1]
    cdef int i # pour optimiser la boucle
    cdef double[::1] Y = np.zeros(N)

    for i in range(N):
        point = sorted_scores[i]
        d = cross_prod_vect(first_point,last_point,point)
        Y[i]=abs(d)
    print(Y)
    if np.max(Y)>0:
        x_elbow=sorted_scores[np.argmax(Y)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow


# measure time just for the cythonised function

In [10]:
#vérification que les résultats soient bons pour un exemple assez grand
D = np.random.rand(10**3,)
final_array = np.array([i for i in enumerate(D)])
elbow_python(final_array)[0] ==naive_cython_elbow(final_array)[0] ==cdef_elbow(final_array)[0] ==c_elbow_memoryviews(final_array)[0] ==c_elbow_noboundscheck_nowraparound(final_array)[0] ==c_elbow_continous(final_array)[0]

True

In [11]:
from timeit import timeit
from tqdm import tqdm
array_len_range= range(1,7)
functions = ['elbow_python','naive_cython_elbow','cdef_elbow','c_elbow_memoryviews','c_elbow_noboundscheck_nowraparound','c_elbow_continous']
functions_arrays=[(i,[]) for i in functions]

for i in tqdm(array_len_range):
    num_iter = 5-min(i,3)
    D = np.random.rand(10**i,)

    for function,array in functions_arrays:

        final_array = np.array([i for i in enumerate(D)])
        time = timeit(str(function + '(final_array)'), number=num_iter, globals=globals())/num_iter
        array.append(time)


 67%|██████▋   | 4/6 [00:04<00:02,  1.15s/it]


KeyboardInterrupt: 

In [ ]:

from matplotlib import pyplot as plt
import numpy as np


plt.style.use(['seaborn-poster','fivethirtyeight'])

# plt.style.use(['dark_background', 'presentation'])
x = [10**i for i in array_len_range]

fig, ax = plt.subplots()
for function,array in functions_arrays:
    ax.plot(x, array,label=function)

ax.set_title("Analyse des temps d'execution (fonction elbow)")
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')

plt.xlabel ("longueur de l array")
plt.ylabel ("temps d'execution (s)")
plt.show()

In [27]:
## sur un échantillon réduit

In [13]:
from timeit import timeit
elbow_c=[]
array_len_range= range(10,1000,25)
functions = ['elbow_python','naive_cython_elbow','cdef_elbow','c_elbow_memoryviews','c_elbow_noboundscheck_nowraparound','c_elbow_continous']
functions_arrays=[(i,[]) for i in functions]
for i in array_len_range:
    num_iter = 10
    D = np.random.rand(i,)

    for function,array in functions_arrays:
#         print(i)

        final_array = np.array([i for i in enumerate(D)])
        time = timeit(str(function + '(final_array)'), number=num_iter, globals=globals())/num_iter
        array.append(time)

In [1]:

from matplotlib import pyplot as plt
import numpy as np


plt.style.use(['seaborn-poster','fivethirtyeight'])

# plt.style.use(['dark_background', 'presentation'])
# x = [10**i for i in array_len_range]

x = [i for i in array_len_range]


fig, ax = plt.subplots()
for function,array in functions_arrays:
    ax.plot(x, array,label=function)

ax.set_title("Analyse des temps d'execution (fonction elbow)")
ax.legend(loc="lower right")

ax.set_yscale('log')
# ax.set_xscale('log')

plt.xlabel("longueur de l array")
plt.ylabel("temps d'execution (s)")
plt.show()

NameError: name 'array_len_range' is not defined

# prange

In [51]:
%%cython  --compile-args=-fopenmp  --link-args=-fopenmp
# cython: infer_types=True, boundscheck=False 
from cython.parallel import prange
from libc.math cimport sin
import numpy as np
cimport numpy as np
import numpy as np
from libc.math cimport sqrt
import networkx as nx
from cpython cimport array
from array import array as pyarray
from libc.string cimport memcpy
import cython
from cython.parallel import prange
cimport openmp as omp


cdef float cross_prod_vect(double[::1] first_point, double[::1] last_point,  double[::1] point):
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef float Bx = (point[0]-first_point[0])
    cdef float By = (point[1]-first_point[1])
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm
    

cdef float cross_prod(float Ax, float Ay, float Bx, float By):
    cdef float result= Ax*By-Ay*Bx
    cdef float norm = sqrt(Ax*Ax+Ay*Ay)
    return result/norm

cdef float density_with_e_v(float e, float v):
    return abs(e)/(abs(v)*(abs(v)-1))   

# Multi-threading version
cpdef double[:] px3(double[:, ::1] a):
    cdef Py_ssize_t i
    cdef int n = a.size, j
    cdef double[::1] first_point = a[0]
    cdef double[::1] last_point = a[n-1]

    cdef double[::1] result = np.empty(n, dtype=np.float64)
    
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef vector[double] v=[]
  #  cdef omp.omp_lock_t lock
 #   omp.omp_init_lock(&lock)
    for i in prange(n, nogil=True,schedule='static', chunksize=1):
#         omp.omp_set_lock(&lock)
        v[i] = Ax*(a[i][1]-first_point[1])-Ay*(a[i][0]-first_point[0])/sqrt(Ax*Ax+Ay*Ay)
#        omp.omp_unset_lock(&lock)
    return result

# cython: infer_types=True, infer_types.verbose=True, boundscheck=False 
# from cython.parallel import prange
# from libcpp.vector cimport vector
# cdef pappend():
#     cdef Py_ssize_t i
#     cdef vector[long] v=[]
#     for i in prange(10, nogil=True, num_threads = 4):
#         #with gil:
#             v.push_back(i)    
#     return v



@cython.boundscheck(False)
@cython.wraparound(False)
cpdef c_elbow_continous_parallel(double[:, ::1] sorted_scores):

    cdef int elbow_point
    cdef int N = sorted_scores.shape[0]
    cdef double[::1] first_point = sorted_scores[0]
    cdef double[::1] last_point = sorted_scores[N-1]
    cdef int i # pour optimiser la boucle
    cdef double[:] Y = px3(sorted_scores) #np.zeros(N)
    print(np.array(Y))
#     cdef double[:] resutl = 
#     for i in range(N):
#         point = sorted_scores[i]
#         d = cross_prod_vect(first_point,last_point,point)
#         Y[i]=abs(d)
        
    if np.max(Y)>0:
        x_elbow=sorted_scores[np.argmax(Y)]
    else :
        x_elbow=sorted_scores[0]

    return x_elbow



Error compiling Cython file:
------------------------------------------------------------
...

    cdef double[::1] result = np.empty(n, dtype=np.float64)
    
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef vector[double] v=[]
        ^
------------------------------------------------------------

/home/jaime/.cache/ipython/cython/_cython_magic_84789ccb342c427eff8fa79b63bf7ca3.pyx:46:9: 'vector' is not a type identifier

Error compiling Cython file:
------------------------------------------------------------
...

    cdef double[::1] result = np.empty(n, dtype=np.float64)
    
    cdef float Ax = (last_point[0] - first_point[0])
    cdef float Ay = (last_point[1]-first_point[1])
    cdef vector[double] v=[]
                         ^
------------------------------------------------------------

/home/jaime/.cache/ipython/cython/_cython_magic_84789ccb342c427eff8fa79b63bf7ca3.pyx:46:26: Cannot coerce list to type '<error

TypeError: object of type 'NoneType' has no len()

In [47]:
D = np.random.rand(10,)
final_array = np.array([i for i in enumerate(D)])
c_elbow_continous_parallel(final_array)[0]
c_elbow_continous(final_array)[0]

[0.00000000e+000 1.00000000e+000 2.00000000e+000 3.00000000e+000
 4.00000001e+000 5.00000001e+000 6.00000001e+000 7.00000001e+000
 8.00000001e+000 9.00000001e+000 7.84714045e-311 6.95152907e-310
 1.58101007e-322 0.00000000e+000 1.58101007e-322 9.88131292e-324
             nan 8.30030285e-322 0.00000000e+000 0.00000000e+000]
<MemoryView of 'ndarray' object>


7.0

In [112]:
%%time
import numpy as np
a = np.arange(10000, dtype=float)
print(a)
px3(a)

[0.000e+00 1.000e+00 2.000e+00 ... 9.997e+03 9.998e+03 9.999e+03]


ValueError: Buffer has wrong number of dimensions (expected 2, got 1)

In [58]:
%%time
import numpy as np
a = np.arange(10000, dtype=float)
print(a)
x3(a)

[0.000e+00 1.000e+00 2.000e+00 ... 9.997e+03 9.998e+03 9.999e+03]
CPU times: user 1.68 s, sys: 24 µs, total: 1.68 s
Wall time: 1.67 s


<MemoryView of 'ndarray' at 0x7f47d7208540>

SyntaxError: invalid syntax (<ipython-input-59-3d8539c82216>, line 3)

## time

In [15]:
# G = nx.duplication_divergence_graph(3000,0.7)

In [16]:
# %timeit get_keywords(G)

In [17]:
# %timeit c_get_keywords(G)

## memory usage

In [18]:
# pip install memory_profiler

In [19]:
# %load_ext memory_profiler

In [20]:
# %memit get_keywords(g)

In [21]:
# %memit c_get_keywords(g)

# Profiler to prioritaze parts to cythonise

In [22]:
# import cProfile
# import re


In [23]:
# cProfile.run('re.compile("foo|bar")')